In [ ]:
from CLOGP import MGmB, calculo_S, batche, calcula_p_chapeu, calculo_accuracy, calcular_erro, sigmoid, sinal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import seaborn as sns
# carregar dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
def exhaustive_code(k):
    segment_length = 2**(k-1) - 1
    codes = []
    # Vamos admitir que irá sempre receber k=4
    codes.append([1]*segment_length)

    row = []
    segment_length = 2**(k-2)
    row.extend([0] * segment_length)
    row.extend([1] * (segment_length - 1))
    codes.append(row)
    
    row = []
    segment_length = 2**(k-3)
    row.extend([0] * segment_length)
    row.extend([1] * segment_length)
    row.extend([0] * segment_length)
    row.extend([1] * (segment_length - 1))
    codes.append(row)

    codes.append([0,1,0,1,0,1,0])
    
    return codes

k = 4
code = exhaustive_code(k)
for row in code:
    print(row)

In [ ]:
# ficar só os 100 primeiros de cada classe
def select_first_n_per_class(x, y, n):
    x_selected = []
    y_selected = []
    
    for digit in np.unique(y):
        idx = np.where(y == digit)[0][:n]
        x_selected.append(x[idx])
        y_selected.append(y[idx])
    
    return np.concatenate(x_selected), np.concatenate(y_selected)